In [19]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown

In [20]:
from io import StringIO
import os

In [1]:
#start spark session
import findspark
findspark.init()
from pyspark import SparkContext
sc=SparkContext()
sc

<SparkContext master=yarn appName=pyspark-shell>

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()
spark

In [2]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [21]:
s3 = boto3.resource('s3')
s3

s3.ServiceResource()

In [22]:
bucket = s3.Bucket(name="noaa-global-hourly-pds")
bucket

s3.Bucket(name='noaa-global-hourly-pds')

In [ ]:
hadoop fs -cp s3://noaa-global-hourly-pds/1901/ s3://buckets/noaa-502/

## Get NameList

In [15]:
srdd2=spark.read.format("csv").option("header", "true").load("s3://noaa-global-hourly-pds/2019/*.csv")

In [33]:
rdd1 = sc.textFile("s3://open-data-noaa/1970/*.csv")

In [35]:
rdd1

s3://open-data-noaa/1970/*.csv MapPartitionsRDD[74] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
srdd3.take(2)

[Row(STATION='02907099999', DATE='1901-01-01T06:00:00', SOURCE='4', LATITUDE='64.3333333', LONGITUDE='23.45', ELEVATION='5.0', NAME='KALAJOKI ULKOKALLA, FI', REPORT_TYPE='FM-12', CALL_SIGN='99999', QUALITY_CONTROL='V020', WND='270,1,N,0159,1', CIG='99999,9,9,N', VIS='000000,1,N,9', TMP='-0078,1', DEW='+9999,9', SLP='10200,1', GF1='08,99,1,99,9,99,9,99999,9,99,9,99,9', MW1=None),
 Row(STATION='02907099999', DATE='1901-01-01T13:00:00', SOURCE='4', LATITUDE='64.3333333', LONGITUDE='23.45', ELEVATION='5.0', NAME='KALAJOKI ULKOKALLA, FI', REPORT_TYPE='FM-12', CALL_SIGN='99999', QUALITY_CONTROL='V020', WND='290,1,N,0082,1', CIG='99999,9,9,N', VIS='000000,1,N,9', TMP='-0072,1', DEW='+9999,9', SLP='10200,1', GF1='04,99,1,99,9,99,9,99999,9,99,9,99,9', MW1=None)]

In [54]:
srdd2.count()

135944480

In [8]:
srdd.count()

6565

In [17]:
srdd

DataFrame[STATION: string, DATE: string, SOURCE: string, LATITUDE: string, LONGITUDE: string, ELEVATION: string, NAME: string, REPORT_TYPE: string, CALL_SIGN: string, QUALITY_CONTROL: string, WND: string, CIG: string, VIS: string, TMP: string, DEW: string, SLP: string, GF1: string, MW1: string]

In [23]:
prefix = "1901/"
FilesNotFound = True
FileNameList=[]
for obj in bucket.objects.filter(Prefix=prefix):
    FileNameList.append('{0}{1}/{2}'.format('s3://',bucket.name, obj.key))
    print('{0}/{1}'.format(bucket.name, obj.key))
    FilesNotFound = False
if FilesNotFound:
     print("ALERT", "No file in {0}/{1}".format(bucket, prefix))

FileNameList

noaa-global-hourly-pds/1901/02907099999.csv
noaa-global-hourly-pds/1901/02950099999.csv
noaa-global-hourly-pds/1901/02960099999.csv
noaa-global-hourly-pds/1901/02972099999.csv
noaa-global-hourly-pds/1901/02981099999.csv
noaa-global-hourly-pds/1901/22707099999.csv


['s3://noaa-global-hourly-pds/1901/02907099999.csv',
 's3://noaa-global-hourly-pds/1901/02950099999.csv',
 's3://noaa-global-hourly-pds/1901/02960099999.csv',
 's3://noaa-global-hourly-pds/1901/02972099999.csv',
 's3://noaa-global-hourly-pds/1901/02981099999.csv',
 's3://noaa-global-hourly-pds/1901/22707099999.csv']

In [44]:
FileNameList[1]

's3://noaa-global-hourly-pds/1901/02950099999.csv'

In [24]:
rdd = sc.textFile(FileNameList[1])
#rdd = sc.textFile("s3://noaa-global-hourly-pds/1931/12195099999.csv")

In [46]:
rdd.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1"',
 '"02950099999","1901-01-01T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","140,1,N,0041,1","99999,9,9,N","000000,1,N,9","-0144,1","+9999,9","10271,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T13:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0139,1","99999,9,9,N","000000,1,N,9","-0100,1","+9999,9","10259,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T20:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0159,1","99999,9,9,N","000000,1,N,9","-0067,1","+9999,9","10239,1","05,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-02T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","250,1,N,0139,1","99999,9,9,N"

In [54]:
rdd.count()

1096

In [70]:
rdd.union(rdd2).count()

2192

In [80]:
len(FileNameList)
range(1,len(FileNameList))

range(1, 6)

In [36]:
i=1
rdd_all=rdd
for i in range(1,len(FileNameList)):
    print(FileNameList[i])
    rdd_sep=sc.textFile(FileNameList[i])
    rdd_all=rdd_all.union(rdd_sep)
    
rdd_sep
rdd_all.count()
    
    

s3://noaa-global-hourly-pds/1901/02950099999.csv
s3://noaa-global-hourly-pds/1901/02960099999.csv
s3://noaa-global-hourly-pds/1901/02972099999.csv
s3://noaa-global-hourly-pds/1901/02981099999.csv
s3://noaa-global-hourly-pds/1901/22707099999.csv


6571

In [85]:
rdd_all.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1"',
 '"02950099999","1901-01-01T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","140,1,N,0041,1","99999,9,9,N","000000,1,N,9","-0144,1","+9999,9","10271,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T13:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0139,1","99999,9,9,N","000000,1,N,9","-0100,1","+9999,9","10259,1","08,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-01T20:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","200,1,N,0159,1","99999,9,9,N","000000,1,N,9","-0067,1","+9999,9","10239,1","05,99,1,99,9,99,9,99999,9,99,9,99,9"',
 '"02950099999","1901-01-02T06:00:00","4","60.1166666","21.7","4.5","PARAINEN FAGERHOLM, FI","FM-12","99999","V020","250,1,N,0139,1","99999,9,9,N"

In [39]:
rdd_all.repartition(1).saveAsTextFile("s3://noaa-502/2/1901.csv")

In [40]:
df=spark.read.format("csv").option("header", "true").load("s3://noaa-502/2/1901.csv")

In [41]:
df

DataFrame[STATION: string, DATE: string, SOURCE: string, LATITUDE: string, LONGITUDE: string, ELEVATION: string, NAME: string, REPORT_TYPE: string, CALL_SIGN: string, QUALITY_CONTROL: string, WND: string, CIG: string, VIS: string, TMP: string, DEW: string, SLP: string, GF1: string]

In [42]:
df.write.csv("s3://noaa-502/3", sep=',', header=True,mode="overwrite")

In [45]:
import random
desired_pct = 10
#file_sample = random.sample(FileNameList, int(len(file_list) * desired_pct / 100))
file_sample = random.sample(FileNameList, 2)
len(file_sample)
file_sample

['s3://noaa-global-hourly-pds/1901/02981099999.csv',
 's3://noaa-global-hourly-pds/1901/02950099999.csv']

In [47]:
i=0
sample_rdd = sc.emptyRDD()
for i in range(0,len(file_sample)):
    print(file_sample[i])
    rdd_sep=sc.textFile(file_sample[i])
    sample_rdd=sample_rdd.union(rdd_sep)
    
rdd_sep
rdd_all.count()

s3://noaa-global-hourly-pds/1901/02981099999.csv
s3://noaa-global-hourly-pds/1901/02950099999.csv


6571

## Try Data 2019

In [48]:
prefix = "2019/"
FilesNotFound = True
FileName2019=[]
for obj in bucket.objects.filter(Prefix=prefix):
    FileName2019.append('{0}{1}/{2}'.format('s3://',bucket.name, obj.key))
    #print('{0}/{1}'.format(bucket.name, obj.key))
    FilesNotFound = False
if FilesNotFound:
     print("ALERT", "No file in {0}/{1}".format(bucket, prefix))

len(FileName2019)

6

In [49]:
len(FileName2019)

13883

In [84]:
#Sample
import random
desired_pct = 10
file_sample_2019 = random.sample(FileName2019, int(len(FileName2019) * desired_pct / 800))
#file_sample_2019 = random.sample(FileNameList, 2)
len(file_sample_2019)

173

In [85]:
#Merge
i=0
sample_rdd2019 = sc.emptyRDD()
for i in range(0,len(file_sample_2019)):
    #print(file_sample[i])
    rdd_sep=sc.textFile(file_sample_2019[i])
    sample_rdd2019=sample_rdd2019.union(rdd_sep)
    
#rdd_sep


In [86]:
sample_rdd2019

UnionRDD[30113] at union at <unknown>:0

In [71]:
#sample_rdd2019.count()

In [68]:
rdd_all

UnionRDD[90] at union at NativeMethodAccessorImpl.java:0

In [55]:
rdd_sep

s3://noaa-global-hourly-pds/2019/64222099999.csv MapPartitionsRDD[11255] at textFile at <unknown>:0

In [87]:
#Save rdd
sample_rdd2019.repartition(1).saveAsTextFile("s3://open-data-noaa/2019sample")

In [88]:
#Save as csv
df=spark.read.format("csv").option("header", "true").load("s3://open-data-noaa/2019sample")

In [89]:
df.write.csv("s3://open-data-noaa/2019samples", sep=',', header=True,mode="overwrite")

**delete header rows**

In [84]:
# 对上述结果筛选掉一些常见的介词，如
# header_words = ['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1","MW1"']
# clean_rdd = rdd_all.filter(lambda w: w not in header_words)
# clean_rdd.count()

6570

In [92]:
data = spark.read\
  .format('csv')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load("s3://open-data-noaa/2019samples")

In [100]:
data1 = spark.read\
  .format('csv')\
  .option('header', 'false')\
  .option('inferSchema', 'true')\
  .load("s3://open-data-noaa/2019samples")

In [102]:
data1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [103]:
data1.show(5)

+-----------+-------------------+---+----------+-----------+-----+--------------------+-----+-----+----+--------------+-----------+------------+---------+---------+-------+-----------+-----------+-----------+----+----+------------------+------------------+------------------+---------------+--------------------+--------------------+----+--------------------+----+----------------+----+----+--------------------+----+
|        _c0|                _c1|_c2|       _c3|        _c4|  _c5|                 _c6|  _c7|  _c8| _c9|          _c10|       _c11|        _c12|     _c13|     _c14|   _c15|       _c16|       _c17|       _c18|_c19|_c20|              _c21|              _c22|              _c23|           _c24|                _c25|                _c26|_c27|                _c28|_c29|            _c30|_c31|_c32|                _c33|_c34|
+-----------+-------------------+---+----------+-----------+-----+--------------------+-----+-----+----+--------------+-----------+------------+---------+---------+

In [110]:
data1.count()

1426323

In [111]:
data1.where('_c0=="STATION"').count()

173

In [112]:
data2=data1.where('_c0!="STATION"')

In [113]:
data2.count()

1426150

In [148]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SQLContext, Row, DataFrame, Column}
import org.apache.spark.ml.feature.VectorAssembler

SyntaxError: invalid syntax (<ipython-input-148-26a74203e16a>, line 3)

In [159]:
from pyspark.sql import SparkSession
#from pyspark.sql import functions._
#import pyspark.spark.sql.{SQLContext, Row, DataFrame, Column}
from pyspark.sql import Row,DataFrame,Column
from pyspark.sql import SQLContext
#from pyspark.ml import feature.VectorAssembler

In [160]:
newColumns = Seq("STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE", "CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1")
#val data2 = data2.toDF(newColumns:_*)

NameError: name 'Seq' is not defined

In [ ]:
newColumns

In [118]:
data2.withColumnRenamed("STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE", "CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1")\
.printSchema()

TypeError: withColumnRenamed() takes 3 positional arguments but 18 were given

In [149]:
import spark.implicits._

ModuleNotFoundError: No module named 'spark'

In [98]:
import spark.implicits._

data1=seq(data).toDF("STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1")
#"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","GF1"

ModuleNotFoundError: No module named 'spark'

In [ ]:
spark = SparkSession.builder.getOrCreate()

columns = ['id', 'dogs', 'cats']
vals = [(1, 2, 0), (2, 0, 1)]

df = spark.createDataFrame(vals, columns)

newRow = spark.createDataFrame([(4,5,7)], columns)
appended = df.union(newRow)
appended.show()

In [ ]:
#Convert Data Type

In [141]:
data2_converted=data2.withColumn("_c31", data2["_c31"].cast("float"))

In [142]:
data2_converted

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string, _c15: string, _c16: string, _c17: string, _c18: string, _c19: string, _c20: string, _c21: string, _c22: string, _c23: string, _c24: string, _c25: string, _c26: string, _c27: string, _c28: string, _c29: string, _c30: string, _c31: float, _c32: string, _c33: string, _c34: string]

In [145]:
#按station操作
#https://sparkbyexamples.com/spark/using-groupby-on-dataframe/
data2_converted.groupBy("_c0").sum("_c31").show()


+-----------+---------+
|        _c0|sum(_c31)|
+-----------+---------+
|52818099999|     null|
|99404199999|     null|
|94367099999|     null|
|72253512909|     null|
|44305099999|     null|
|71136099999|     null|
|81908099999|     null|
|26657099999|     null|
|43344099999|     null|
|07460099999|     null|
|72656604965|     null|
|59046099999|     null|
|95146099999|     null|
|68375099999|     null|
|74778099999|     null|
|11022099999|     null|
|83424099999|     null|
|41933099999|     null|
|42871099999|     null|
|15247099999|     null|
+-----------+---------+
only showing top 20 rows



In [147]:
data2["_c31"].show(20)

TypeError: 'Column' object is not callable

In [128]:
#Drop duplicate rows in pyspark by a specific column
data2_dropdup=data2.dropDuplicates((['_c0']))

In [130]:
data2_dropdup.count()

173

In [136]:
data2_dropdup.show(2)

+-----------+-------------------+---+----------+------+------+-------------------+-----+-----+----+--------------+-----------+------------+-------+-------+-------+----------------+---------------+------+--------------------+----+----+----+----+----+----+--------------------+----+----+----+---------------+----------------+----+----+----+
|        _c0|                _c1|_c2|       _c3|   _c4|   _c5|                _c6|  _c7|  _c8| _c9|          _c10|       _c11|        _c12|   _c13|   _c14|   _c15|            _c16|           _c17|  _c18|                _c19|_c20|_c21|_c22|_c23|_c24|_c25|                _c26|_c27|_c28|_c29|           _c30|            _c31|_c32|_c33|_c34|
+-----------+-------------------+---+----------+------+------+-------------------+-----+-----+----+--------------+-----------+------------+-------+-------+-------+----------------+---------------+------+--------------------+----+----+----+----+----+----+--------------------+----+----+----+---------------+----------------

In [131]:
# Drop rows with NA or missing values in pyspark
data2_dropna=data2_dropdup.dropna()
#似乎没必要dropna

In [132]:
data2_dropna.count()

1

In [134]:
data2_dropna.show()

+-----------+-------------------+---+-----+----------+---+-------------+-----+-----+----+--------------+-----------+------------+---------+---------+-------+-----------+--------+--------+----+----+------------------+------------------+------------------+----+--------------------+--------------------+----+----+----+----------------+----+----+--------------------+----+
|        _c0|                _c1|_c2|  _c3|       _c4|_c5|          _c6|  _c7|  _c8| _c9|          _c10|       _c11|        _c12|     _c13|     _c14|   _c15|       _c16|    _c17|    _c18|_c19|_c20|              _c21|              _c22|              _c23|_c24|                _c25|                _c26|_c27|_c28|_c29|            _c30|_c31|_c32|                _c33|_c34|
+-----------+-------------------+---+-----+----------+---+-------------+-----+-----+----+--------------+-----------+------------+---------+---------+-------+-----------+--------+--------+----+----+------------------+------------------+------------------+----+-

In [93]:
data.show(5)

+-----------+-------------------+---+----------+-----------+-----+--------------------+-----+-----+----+--------------+-----------+------------+---------+---------+-------+-----------+-----------+-----------+----+----+------------------+------------------+------------------+---------------+----------------------+-----------------------------------+----+--------------------+----+----------------+----+----+----------------------------------------------------------------------+----+
|43314099999|2019-07-10T00:00:00|  4|     11.25| 75.7833333|  5.0|       KOZHIKODE, IN|FM-12|99999|V020|999,9,C,0000,1|99999,9,9,N|004000,1,9,9|+0228,113|+0228,114|10078,1|21,0800,3,1|   6,1,02,1|   2,1,02,1|_c19|_c20|03,1,+00800,1,06,1|99,9,+99999,9,03,1|99,9,+99999,9,00,1|           _c24|9,MSL   ,+99999,+99999|07,99,1,03,1,99,9,00800,1,99,9,99,9|_c27|                _c28|_c29|9,9,999,9,-005,1|02,1|_c32|SYN06443314 21596 70000 10228 20228 40078 70262 83472 333 59005 6080/=|_c34|
+-----------+-----------------

In [95]:
data.printSchema()

root
 |-- 43314099999: string (nullable = true)
 |-- 2019-07-10T00:00:00: string (nullable = true)
 |-- 4: string (nullable = true)
 |-- 11.25: string (nullable = true)
 |-- 75.7833333: string (nullable = true)
 |-- 5.0: string (nullable = true)
 |-- KOZHIKODE, IN: string (nullable = true)
 |-- FM-12: string (nullable = true)
 |-- 99999: string (nullable = true)
 |-- V020: string (nullable = true)
 |-- 999,9,C,0000,1: string (nullable = true)
 |-- 99999,9,9,N: string (nullable = true)
 |-- 004000,1,9,9: string (nullable = true)
 |-- +0228,113: string (nullable = true)
 |-- +0228,114: string (nullable = true)
 |-- 10078,1: string (nullable = true)
 |-- 21,0800,3,1: string (nullable = true)
 |-- 6,1,02,1: string (nullable = true)
 |-- 2,1,02,1: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- 03,1,+00800,1,06,1: string (nullable = true)
 |-- 99,9,+99999,9,03,1: string (nullable = true)
 |-- 99,9,+99999,9,00,1: string (nullable = true)


In [96]:
data.describe()

DataFrame[summary: string, 43314099999: string, 2019-07-10T00:00:00: string, 4: string, 11.25: string, 75.7833333: string, 5.0: string, KOZHIKODE, IN: string, FM-12: string, 99999: string, V020: string, 999,9,C,0000,1: string, 99999,9,9,N: string, 004000,1,9,9: string, +0228,113: string, +0228,114: string, 10078,1: string, 21,0800,3,1: string, 6,1,02,1: string, 2,1,02,1: string, _c19: string, _c20: string, 03,1,+00800,1,06,1: string, 99,9,+99999,9,03,1: string, 99,9,+99999,9,00,1: string, _c24: string, 9,MSL   ,+99999,+99999: string, 07,99,1,03,1,99,9,00800,1,99,9,99,9: string, _c27: string, _c28: string, _c29: string, 9,9,999,9,-005,1: string, 02,1: string, _c32: string, SYN06443314 21596 70000 10228 20228 40078 70262 83472 333 59005 6080/=: string, _c34: string]

## Merge Data

In [47]:
rdd

s3://noaa-global-hourly-pds/1901/02950099999.csv MapPartitionsRDD[8] at textFile at NativeMethodAccessorImpl.java:0

In [2]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')

In [3]:
# def create_bucket(bucket):
#     import logging

#     try:
#         s3.create_bucket(Bucket=bucket)
#     except botocore.exceptions.ClientError as e:
#         logging.error(e)
#         return 'Bucket ' + bucket + ' could not be created.'
#     return 'Created or already exists ' + bucket + ' bucket.'

In [4]:
#create_bucket('open-data-noaa')

ERROR:root:An error occurred (BucketAlreadyExists) when calling the CreateBucket operation: The requested bucket name is not available. The bucket namespace is shared by all users of the system. Please select a different name and try again.


'Bucket open-data-noaa could not be created.'

In [5]:
def list_buckets(match=''):
    response = s3.list_buckets()
    if match:
        print(f'Existing buckets containing "{match}" string:')
    else:
        print('All existing buckets:')
    for bucket in response['Buckets']:
        if match:
            if match in bucket["Name"]:
                print(f'  {bucket["Name"]}')

In [15]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': '4C6DF56D8448AF7D',
  'HostId': 'T3vPu5Ivo2fIB6c/BLDjag3t5nFvkRt84sbwzE6uka1RUacCp2YvkemyxPNCJo/gHL6jEKMEilU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'T3vPu5Ivo2fIB6c/BLDjag3t5nFvkRt84sbwzE6uka1RUacCp2YvkemyxPNCJo/gHL6jEKMEilU=',
   'x-amz-request-id': '4C6DF56D8448AF7D',
   'date': 'Sun, 19 Apr 2020 22:04:00 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': '502hw4',
   'CreationDate': datetime.datetime(2020, 3, 27, 1, 29, 29, tzinfo=tzlocal())},
  {'Name': '502lab3m233',
   'CreationDate': datetime.datetime(2020, 1, 30, 16, 38, 49, tzinfo=tzlocal())},
  {'Name': 'aws-logs-270109529444-us-east-1',
   'CreationDate': datetime.datetime(2020, 1, 29, 1, 15, 55, tzinfo=tzlocal())},
  {'Name': 'noaa-502',
   'CreationDate': datetime.datetime(2020, 4, 19, 21, 52, 17, tzinfo=tzlocal())}],
 'Owner': {'DisplayName': 'awslabsc0w518423t157519000

In [14]:
list_buckets(match='99')

Existing buckets containing "99" string:


In [7]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Matched file size is {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Bucket {bucket} total size is {total_size_gb/1024:3.1f}GB with {total_files} files')

In [ ]:
list_bucket_contents(bucket='noaa-isd-pds', match = 'data')

In [ ]:
list_bucket_contents(bucket='noaa-global-hourly-pds', match = '1950')

In [41]:
rdd = sc.textFile("s3://noaa-global-hourly-pds/1931/12195099999.csv")

In [42]:
rdd.take(5)

['"STATION","DATE","SOURCE","LATITUDE","LONGITUDE","ELEVATION","NAME","REPORT_TYPE","CALL_SIGN","QUALITY_CONTROL","WND","CIG","VIS","TMP","DEW","SLP","AA1","AY1","GF1","IA1","KA1","MD1","MW1","EQD"',
 '"12195099999","1931-01-09T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","999,1,C,0000,1","00450,1,C,N","002000,1,N,9","-0050,1","-0056,1","10266,1",,"2,1,99,9","99,99,9,08,1,05,1,00450,1,00,1,00,1",,,,,"Q01+000032APCTENQ02+000002APC3  "',
 '"12195099999","1931-01-10T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","200,1,N,0010,1","00450,1,C,N","002000,1,N,9","-0061,1","-0067,1","10298,1",,"2,1,99,9","08,99,1,08,1,05,1,00450,1,00,1,00,1",,,"3,1,002,1,+999,9",,',
 '"12195099999","1931-01-13T12:00:00","4","54.1333333","22.95","186.0","SUWALKI, PL","FM-12","99999","V020","180,1,N,0010,1","00780,1,C,N","004000,1,N,9","-0022,1","-0028,1","10111,1",,"2,1,99,9","08,99,1,08,1,05,1,00800,1,00,1,00,1",,,"8,1,004,1,+999,9",,',
 '"1219